# 0. Installing Transformers and Importing Dependencies

In [5]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# 1. Load Summarization Pipeline

In [2]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

# 2. Get The Feedback Raw Data

In [59]:
feedback_data = pd.read_excel(r'/Users/neerajkatiyar/Documents-HD/OSE-SciLearn/SciLearn-NLP/Data/feedback.xlsx')
merged_data =pd.read_excel(r'/Users/neerajkatiyar/Documents-HD/OSE-SciLearn/SciLearn-NLP/Data/Merged_feedback.xlsx')

In [60]:
feedback_data['summary']= np.nan
feedback_data['original_length']= np.nan
feedback_data['summary_length']= np.nan

In [61]:
ARTICLE = feedback_data['Text'][0]

In [64]:
ARTICLE.split()

191

# 3. Chunk Text

In [65]:
max_chunk = 500

In [15]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [16]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [17]:
len(chunks)

1

# 4. Summarize Text

In [29]:
res = summarizer(chunks, max_length=200, min_length=80, do_sample=False)

In [46]:
res

[{'summary_text': " I think the most important thing I learned at the bootcamp is to adjust my lifestyle and studying habits to be more effective . Normally I spend long sessions studying and if I do take a break, I watch a TV show or play a game . I'm going to try to increase my physical activity overall (in terms of both exercise and trying to walk to places more than drive) to see how well that helps with my ability to focus throughout a day ."}]

In [24]:
ARTICLE

'I think the most important thing I learned at the bootcamp is to adjust my lifestyle and studying habits\nto be more effective.<eos> For example, one of the things I’ll apply in the future is the use of specific types of breaks.<eos> Normally I spend long sessions studying and if I do take a break, I watch a TV show or play a game.<eos> So now that I know this is the least effective type of break and learning that the most effective break type is exercise, I’m going to try alternating studying and exercise going forward and check if that’s works better for me.<eos> Another thing that I’m going to apply is occasional meditation; the science behind the effectiveness of meditation was really surprising for me and I’ll try weekly meditation sessions to see if it improves both how well I study and how well I feel throughout a day.<eos> Finally, I’m going to try to increase my physical activity overall (in terms of both exercise and trying to walk to places\nmore instead of driving) to see 

In [31]:
' '.join([summ['summary_text'] for summ in res])

" I think the most important thing I learned at the bootcamp is to adjust my lifestyle and studying habits to be more effective . Normally I spend long sessions studying and if I do take a break, I watch a TV show or play a game . I'm going to try to increase my physical activity overall (in terms of both exercise and trying to walk to places more than drive) to see how well that helps with my ability to focus throughout a day ."

In [32]:
text = ' '.join([summ['summary_text'] for summ in res])

In [47]:
text

" I think the most important thing I learned at the bootcamp is to adjust my lifestyle and studying habits to be more effective . Normally I spend long sessions studying and if I do take a break, I watch a TV show or play a game . I'm going to try to increase my physical activity overall (in terms of both exercise and trying to walk to places more than drive) to see how well that helps with my ability to focus throughout a day ."

In [66]:
for index, row in feedback_data.iterrows():
    ARTICLE = row['Text']
    feedback_data['original_length'][index] = len(ARTICLE.split())
    ARTICLE = ARTICLE.replace('.', '.<eos>')
    ARTICLE = ARTICLE.replace('?', '?<eos>')
    ARTICLE = ARTICLE.replace('!', '!<eos>')
    sentences = ARTICLE.split('<eos>')
    current_chunk = 0 
    chunks = []
    for sentence in sentences:
        if len(chunks) == current_chunk + 1: 
            if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(sentence.split(' '))
            else:
                current_chunk += 1
                chunks.append(sentence.split(' '))
        else:
            print(current_chunk)
            chunks.append(sentence.split(' '))

    for chunk_id in range(len(chunks)):
        chunks[chunk_id] = ' '.join(chunks[chunk_id])

    res = summarizer(chunks, max_length=140, min_length=40, do_sample=False)
    text = ' '.join([summ['summary_text'] for summ in res])
    length = len(text.split())
    feedback_data['summary'][index] = text
    feedback_data['summary_length'][index] = length
    

<ipython-input-66-f1953e1ec181>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feedback_data['original_length'][index] = len(ARTICLE.split())


0


<ipython-input-66-f1953e1ec181>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feedback_data['summary'][index] = text
<ipython-input-66-f1953e1ec181>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feedback_data['summary_length'][index] = length


0
0
0
0
0
0
0


Your max_length is set to 140, but you input_length is only 5. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)


...

0
0


Your max_length is set to 140, but you input_length is only 123. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


0
0
0
0
0
0
0
0
0


Your max_length is set to 140, but you input_length is only 100. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


0
0
0
0
0
0
0


63

# 5. Output to Excel File

In [90]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)

In [67]:
feedback_data.to_excel('Summarized_feedback.xls')

<ipython-input-67-b479326cd930>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  feedback_data.to_excel('Summarized_feedback.xls')


In [68]:
feedback_data

,Text,summary,original_length,summary_length
0,I think the most important thing I learned at ...,I think the most important thing I learned at...,191.0,63.0
1,"In the SciLearn bootcamp, I learn that partici...","In the SciLearn bootcamp, I learn that partic...",141.0,63.0
2,"During the Scilearn bootcamp, I learned more a...","During the Scilearn bootcamp, I learned more ...",185.0,66.0
3,This boot camp gave me an insight into some di...,This boot camp gave me an insight into some d...,113.0,56.0
4,The Scilearn bootcamp was delivering many sugg...,The Scilearn bootcamp was delivering many sug...,190.0,54.0
5,SciLearn program introduces some atomic habits...,SciLearn program introduces some atomic habit...,152.0,41.0
6,The sci learn workshop discussed using the pow...,The sci learn workshop discussed using the po...,250.0,55.0
7,The SciLearn bootcamp provided me with insight...,The SciLearn bootcamp provided me with insigh...,165.0,59.0
8,Sci Learn:,Sci Learn: Sci Learn. Sci Learn is a weekly f...,2.0,31.0
9,I think my biggest take away from the SciLearn...,I think my biggest take away from the SciLear...,125.0,64.0
